## 指定のディレクトリからデータを読みだして学習に掛ける

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
with open("./t_fal_audio_data/np_data.dat", mode="rb") as f:
    data=np.fromfile(f,dtype=float)

### データは1配列で返されるので書き込み時の状態に変換する

In [3]:
size=2**12 #サンプリングのデータサイズを入力
n=int(len(data)/(size+1))

data=data.reshape((n, size+1))

print(type(data))
print(len(data))
print(len(data[0]))
print(data[0])

<class 'numpy.ndarray'>
26
4097
[0.214      0.11075341 0.15259981 ... 0.02155629 0.05022515 0.15259981]


In [48]:
temp_data=np.array([])
sound_data=np.array([])
for i in range(len(data)):
    temp_data=np.append(temp_data, data[i][0].real)
    sound_data=np.append(sound_data, data[i][1:])

sound_data=sound_data.reshape((n,size))

"""
temp_data:測定温度の配列(float)
sound_data:音声スペクトルデータの2次元配列([[complex]...[]])
"""

    
print(len(sound_data[0]))
print(temp_data)
print(type(temp_data[0]))

4096
[0.214 0.345 0.308 0.346 0.42  0.38  0.431 0.482 0.479 0.501 0.579 0.55
 0.567 0.689 0.636 0.661 0.718 0.759 0.75  0.856 0.795 0.831 0.91  0.886
 0.942 0.94 ]
<class 'numpy.float64'>


<font size=5 color="red">予測データの用意</font>

In [49]:
with open("./t_fal_audio_data/np_prediction_data.dat", mode="rb") as f:
    prediction_data=np.fromfile(f,dtype=float)

prediction_data=prediction_data.reshape((4,4097))

prediciton_temp_data=np.array([])
prediciton_sound_data=np.array([])
for i in range(len(prediction_data)):
    prediciton_temp_data=np.append(prediciton_temp_data, prediction_data[i][0])
    prediciton_sound_data=np.append(prediciton_sound_data, prediction_data[i][1:])
    
prediciton_sound_data=prediciton_sound_data.reshape((4,4096))

print(prediciton_temp_data)
print(prediciton_sound_data)

[33.  49.8 67.  82.1]
[[0.07170207 0.08974837 0.0852675  ... 0.07528725 0.0852675  0.08974837]
 [0.17753448 0.0796667  0.05342869 ... 0.11737094 0.05342869 0.0796667 ]
 [0.0776328  0.02124883 0.03114722 ... 0.03219219 0.03114722 0.02124883]
 [0.01008851 0.07105503 0.08758143 ... 0.03424141 0.08758143 0.07105503]]


# TFのkerasで学習装置を実装してみる

In [50]:
model=keras.Sequential([
    keras.layers.Dense(4096, activation=tf.nn.relu, input_dim=4096),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
#    keras.layers.Dense(10, activation=tf.nn.relu),
    keras.layers.Dense(1)
])

In [51]:
#optimizer = tf.train.RMSPropOptimizer(0.5)
optimizer = tf.train.AdamOptimizer()
 
model.compile(loss='mse',
            optimizer=optimizer,
            metrics=['mae'])

In [52]:
model.fit(sound_data, temp_data, epochs=80, batch_size=20, callbacks=None)

Epoch 1/80
26/26 [==============================] - 1s 46ms/step - loss: 5.5559 - mean_absolute_error: 1.4989
Epoch 2/80
26/26 [==============================] - 0s 7ms/step - loss: 0.2011 - mean_absolute_error: 0.3394
Epoch 3/80
26/26 [==============================] - 0s 7ms/step - loss: 0.1495 - mean_absolute_error: 0.3310
Epoch 4/80
26/26 [==============================] - 0s 7ms/step - loss: 0.1569 - mean_absolute_error: 0.3440
Epoch 5/80
26/26 [==============================] - 0s 7ms/step - loss: 0.0478 - mean_absolute_error: 0.1637
Epoch 6/80
26/26 [==============================] - 0s 7ms/step - loss: 0.1020 - mean_absolute_error: 0.2638
Epoch 7/80
26/26 [==============================] - 0s 7ms/step - loss: 0.0434 - mean_absolute_error: 0.1387
Epoch 8/80
26/26 [==============================] - 0s 7ms/step - loss: 0.0557 - mean_absolute_error: 0.1852
Epoch 9/80
26/26 [==============================] - 0s 7ms/step - loss: 0.0671 - mean_absolute_error: 0.2067
Epoch 10/80
26/26 

26/26 [==============================] - 0s 7ms/step - loss: 5.0628e-06 - mean_absolute_error: 0.0017
Epoch 74/80
26/26 [==============================] - 0s 7ms/step - loss: 3.2288e-06 - mean_absolute_error: 0.0014
Epoch 75/80
26/26 [==============================] - 0s 7ms/step - loss: 4.0659e-06 - mean_absolute_error: 0.0016
Epoch 76/80
26/26 [==============================] - 0s 7ms/step - loss: 6.2043e-06 - mean_absolute_error: 0.0020
Epoch 77/80
26/26 [==============================] - 0s 7ms/step - loss: 8.2294e-06 - mean_absolute_error: 0.0026
Epoch 78/80
26/26 [==============================] - 0s 7ms/step - loss: 1.5259e-05 - mean_absolute_error: 0.0035
Epoch 79/80
26/26 [==============================] - 0s 7ms/step - loss: 3.8058e-06 - mean_absolute_error: 0.0014
Epoch 80/80
26/26 [==============================] - 0s 7ms/step - loss: 2.2039e-06 - mean_absolute_error: 0.0011


In [53]:
test_loss, test_acc = model.evaluate(sound_data, temp_data)

print("Test_loss:", test_loss)
print("Test_accuracy:", test_acc)

26/26 [==============================] - 0s 5ms/step
Test_loss: 1.957693484655465e-06
Test_accuracy: 0.0011329788248986006


In [54]:
test_loss, test_acc = model.evaluate(prediciton_sound_data, prediciton_temp_data)

print("Test_loss:", test_loss)
print("Test_accuracy:", test_acc)

4/4 [==============================] - 0s 5ms/step
Test_loss: 3623.46875
Test_accuracy: 57.38077926635742


## モデルで予想をしてみる

In [55]:
print(model.predict(sound_data)*100)

[[21.373583]
 [34.410004]
 [30.655178]
 [34.513958]
 [41.905914]
 [37.87197 ]
 [42.9814  ]
 [48.15132 ]
 [47.91251 ]
 [50.140495]
 [57.87522 ]
 [54.993313]
 [56.34308 ]
 [68.88635 ]
 [63.479084]
 [66.16797 ]
 [71.68691 ]
 [75.69704 ]
 [74.96135 ]
 [85.445175]
 [79.28791 ]
 [82.88701 ]
 [90.85037 ]
 [88.364944]
 [94.036964]
 [93.91717 ]]


In [57]:
print(model.predict(prediciton_sound_data)*100)
print(prediciton_temp_data)

[[27.718845]
 [56.4799  ]
 [67.59506 ]
 [85.89449 ]]
[33.  49.8 67.  82.1]
